In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.glyphs import Line as Line_glyph, Oval
from bokeh.models import Range1d, LinearAxis, FactorRange, LabelSet, Legend
from bokeh.layouts import gridplot
from bokeh.io import output_file, show

In [2]:
dfPitcher = pd.read_csv("baseballdatabank-master/core/Pitching.csv")
dfBatter = pd.read_csv("baseballdatabank-master/core/Batting.csv")
dfTeams = pd.read_csv("baseballdatabank-master/core/Teams.csv")

dfBatter.tail()
dfBatter = dfBatter.loc[dfBatter["stint"]==1]
dfBatter = dfBatter.loc[dfBatter['G']>20]
dfBatter = dfBatter.loc[dfBatter['AB']>80]
dfBatter = dfBatter.drop(["stint","CS","SO","SH","GIDP"],axis=1).fillna(0)
years = [1870,1880,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010]

dfBatter.head()

,playerID,yearID,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,IBB,HBP,SF
1,addybo01,1871,RC1,0,25,118,30,32,6,0,0,13.0,8.0,4,0.0,0.0,0.0
2,allisar01,1871,CL1,0,29,137,28,40,4,5,0,19.0,3.0,2,0.0,0.0,0.0
3,allisdo01,1871,WS3,0,27,133,28,44,10,2,2,27.0,1.0,0,0.0,0.0,0.0
4,ansonca01,1871,RC1,0,25,120,29,39,11,3,0,16.0,6.0,2,0.0,0.0,0.0
7,barnero01,1871,BS1,0,31,157,66,63,10,9,0,34.0,11.0,13,0.0,0.0,0.0


In [3]:
bestBatPerDec = []
avgBatPerDec = []
extraBest = []
bestYearBat = []

for i in range (0,15):
    lB = (i)*10+1870
    uB = (i)*10+1880
    dfNew = dfBatter.loc[dfBatter["yearID"] < uB]
    dfNew = dfNew.loc[dfNew["yearID"] >=lB]
    bestwOBA = 0
    bestName = ""
    bestTeam = ""
    bestLeag = ""
    bestYear = 0
    accum = 0
    numRows = 0
    bestOthers = ""
    for index,row in dfNew.iterrows():
        numRows += 1
        unintWalk = row["BB"]
        hitByPit = row["HBP"]
        single = row["H"]
        double = row["2B"]
        triple = row["3B"]
        hr = row["HR"]
        sacFly = row["SF"]
        atBats = row["AB"]
        if atBats == 0:
            wOBA = 0
        else:
            wOBA = (.69*unintWalk + .722*hitByPit + .888*single + 1.271*double + 1.616*triple + 2.101*hr)/(atBats+unintWalk+sacFly+hitByPit)
        if wOBA > bestwOBA and atBats >80:
            bestwOBA = wOBA
            bestName = row["playerID"]
            bestTeam = row["teamID"]
            bestYear = row["yearID"]
            bestLeag = row["lgID"]
        
        accum += wOBA
    bestBatPerDec.append([bestwOBA])
    bestYearBat.append(bestYear)

    avgBatPerDec.append(accum/numRows)


In [4]:
dfPitcher = pd.read_csv("baseballdatabank-master/core/Pitching.csv")
dfPitcher = dfPitcher.loc[dfPitcher['stint'] == 1]
dfNewP = dfPitcher.drop(["stint","SV","L","BAOpp","GIDP","SH","SF","SHO","BB","HR","ER"],axis=1).fillna(0)
dfPitcher = dfPitcher[['yearID',"SO",'H','IPouts','BB','HBP','G']].fillna(0)
dfPitcher.head()

,yearID,SO,H,IPouts,BB,HBP,G
0,1871,1,43,78,11,0.0,3
1,1871,13,361,792,37,0.0,30
2,1871,0,8,3,0,0.0,1
3,1871,15,295,639,31,0.0,24
4,1871,0,20,27,3,0.0,1


In [11]:
avgPitcherWHIP = []
bestPitcherWHIP = []
bestPitcherYear = []

for i in range (0,15):
    lB = (i)*10+1870
    uB = (i)*10+1880
    dfNewP = dfPitcher.loc[dfPitcher["yearID"] < uB]
    dfNewP = dfNewP.loc[dfNewP["yearID"] >=lB]
#     dfNewP = dfNewP.loc[dfNewP["G"] > 5]
    avgWHIP = 0
    bestWHIP = 0
    bestYearp = 0
    counter = len(dfNewP)
    for index, row in dfNewP.iterrows():
        H = row['H']
        IP = row["IPouts"]/3
        BB = row['BB']
        WHIP = (BB+H)/IP
        if bestWHIP == 0 or WHIP < bestWHIP:
            bestWHIP = WHIP
            bestYearp = row['yearID']
        
        avgWHIP += WHIP
    if i == 0:
        avgPitcherWHIP.append(0)
    else:
        avgPitcherWHIP.append(avgWHIP/counter)
    bestPitcherWHIP.append(bestWHIP)
    bestPitcherYear.append(bestYearp)

C:\Users\srede\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\srede\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [6]:
output_file("Vis1_Average_Batters_and_Pitcher_vs_Best.html")

batHover = HoverTool(tooltips=[
    ("Year","@x"),
    ("Weighted On-Base Average","@y")    
])

batFig = figure(title="Best Batter Compared to Avg. by Weighted On-Base Average",plot_width = 1000, plot_height = 600, tools=[batHover])
batFig.left[0].formatter.use_scientific = False

batFig.xaxis.axis_label='Year(grouped by decade)'
batFig.yaxis.axis_label="Weighted On-Base Average"

avgBatGraph = batFig.line(years,avgBatPerDec, line_width=1)
avgBatScatt = batFig.scatter(years,avgBatPerDec)
bestBatGraph = batFig.line(bestYearBat, bestBatPerDec,line_width=1,color="red")
bestBatScatt = batFig.scatter(bestYearBat,bestBatPerDec,color="red")

batLegend = Legend(items=[('Avg. Batter',[avgBatGraph]),('Best Batter',[bestBatGraph])])
batFig.add_layout(batLegend)


In [7]:
pitchHover = HoverTool(tooltips=[
    ('Year','@x'),
    ('Walks plus Hits per Inning Pitched(WHIP)', '@y'),
])

pitchFig = figure(title='Best Pitcher Compaired to Avg. by Walks plus Hits per Inning Pitched(WHIP)',plot_width = 1000, plot_height = 600, tools=[pitchHover])
pitchFig.left[0].formatter.use_scientific=False


avgWHIPGraph = pitchFig.line(years, avgPitcherWHIP, line_width=1)
avgWHIPScatter = pitchFig.scatter(years, avgPitcherWHIP)

bestWHIPGraph = pitchFig.line(bestPitcherYear, bestPitcherWHIP, line_width=1, color='red')
bestWHIPScatter = pitchFig.scatter(bestPitcherYear, bestPitcherWHIP, color='red')

pitchLegend = Legend(items=[("Average Pitcher",[avgWHIPGraph]),("Best Pitcher", [bestWHIPGraph])])
pitchFig.add_layout(pitchLegend)


In [12]:
finalGraphs = gridplot([[batFig],[pitchFig]])
show(batFig)
show(pitchFig)
show(finalGraphs)

W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='cdde27e6-fcfc-427e-bd45-7cf12da64f79', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='ef7bedb1-c5d6-49b5-a6ba-da8907bb5f14', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='cdde27e6-fcfc-427e-bd45-7cf12da64f79', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='ef7bedb1-c5d6-49b5-a6ba-da8907bb5f14', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='cdde27e6-fcfc-427e-bd45-7cf12da64f79', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='ef7bedb1-c5d6-49b5-a6ba-da8907bb5f14', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box